## Tarea 1 Pregunta 3

# Node
Definiremos un arbol binario, en particular un arbol de Merkle. Para esto creamos una clase Node que permite almacenar las relaciones de padre e hijo, su posicion respecto a su hermano, ademas del valor del hash en ese nodo.

In [48]:
class Node:

    def __init__(self, string: str):
        self.string = string
        self.parent = None
        self.left_child = None
        self.right_child = None
        self.side = ""
    
    def setParent(self, parent, side) -> bool:
        if self.parent == None:
            self.parent = parent
            self.side = side
            return True
        else:
            return False
    
    def setChild(self, child, side) -> bool:
        if side == "i" and self.left_child == None:
            self.left_child = child
            return True
        elif side == "d" and self.right_child == None:
            self.right_child = child
            return True
        return False
    
    def getCopy(self):
        return Node(self.string)


# MerkleTree

Una vez definida nuestra clase de Node, podemos pasar a definir nuesto arbol binario. El arbol de merkle recibe una lista de strings y una funcion de hash. cada una de las hojas de este arbol es el resultado de utilizar la funcion de hash en cada uno de los strings. Al subir por el arbol, el valor de cada uno de los nodos es el resultado de utilizar la funcion de hash sobre los valores de los hijos concatedados de manera ordenada. Se repite este proceso, repitiendo el ultimo nodo en cada nivel si hace falta, hasta llegar a un solo nodo que es la raiz.

Dentro de esta clase de MerkleTree esta definida una funcion get_proof_for. Esta funcion, recibiendo un string que sea el valor de alguna de las hojas, entrega una lista con el valor de los hermanos de todos los padres de esa hoja hasta llegar a la raiz. Con estos valores se puede verificar que el string entregado es una de las hojas del arbol.

In [49]:
class MerkleTree:

    def __init__(self, strings: list[str], hash_func: callable):
        self.strings = strings
        self.hash = hash_func
        # We setup the leaves of the tree and we start linking them upwards
        self.leaves = [Node(hash_func(s)) for s in strings]
        nodes = self.leaves.copy()
        while len(nodes) != 1:
            nodes = self._setup_level(nodes)
        self._root = nodes[0]
    

    def _setup_level(self, nodes: list[Node]) -> list[Node]:
        if len(nodes) % 2 == 1:
            nodes.append(nodes[-1].getCopy())
        new_nodes = []
        for i in range(len(nodes)):
            if i%2 == 0:
                n1 = nodes[i]
                n2 = nodes[i+1]
                new_string = self.hash(n1.string+n2.string)
                new_node = Node(new_string)
                n1.setParent(new_node, "i")
                n2.setParent(new_node, "d")
                new_node.setChild(n1, "i")
                new_node.setChild(n2, "d")
                new_nodes.append(new_node)
        return new_nodes

    
    def get_root(self) -> str:
        return self._root.string
        
    def get_proof_for(self, item: str) -> (None or list[str]):
        # We check if the item is one of the leaves, otherwise we return None
        leaf = None
        item_hash = self.hash(item)
        for l in self.leaves:
            if l.string == item_hash:
                leaf = l
                break
        if not leaf:
            return None
        # Once we have the leaf, we start constructing the proof we will return
        proof = []
        curr_node = leaf
        while curr_node != self._root:
            parent = curr_node.parent
            if curr_node.side == "i":
                sibling = parent.right_child
            elif curr_node.side == "d":
                sibling = parent.left_child
            proof.append((sibling.string, sibling.side))
            curr_node = parent
        return proof

        


# Verify

Definimos una funcion verify que recibe, un valor a probar, la raiz de un arbol Merkle, una funcion de hash y una lista de valores de forma [value, side], donde value es cualquier string y side es "i" o "d" dependiendo si es el hijo izquierdo o derecho. Con estos parmetros podemos verificar si el valor a probar es efectivamente una de las hojas del arbol. Para esto, utiliza la funcion de hash, con el valor a probar y con los valores de la lista para ir creando una linea de padres, desde la posible hoja que esta probando hasta llegar al valor de la raiz. Si este valor final es igual al valor de la raiz del arbol, entonces el valor a probar si es una de las hojas

In [50]:
def verify(root: str, item: str, proof, hash_func: callable) -> bool:
    prev = hash_func(item)
    for p, side in proof:
        if side == "i":
            new_parent = hash_func(p+prev)
        elif side == "d":
            new_parent = hash_func(prev+p)
        prev = new_parent
    
    if prev == root:
        return True
    return False

# Un Ejemplo

Para probar el codigo, definimos una simple funcion de hash que divide un string por la mitad y lo reordena. Además creamos un pequeño arbol de merkle de prueba. Probaremos tres casos:

In [51]:
# This hash function flips the string in the middle
def test_hash(string: str) -> str:
    middle = len(string)//2
    return string[middle:] + string[0:middle]


In [52]:
leaves = ["hello", "goodbye", "salutations", "impossible", "truth", "false", "boolean"]
MT = MerkleTree(leaves, test_hash)

Caso 1, valor (truth) es una hoja y la prueba es valida

In [53]:
is_truth_a_leaf = verify(MT.get_root(), "truth", MT.get_proof_for("truth"), test_hash)
print(is_truth_a_leaf)

True


Caso 2, valor (truth) es una hoja y la prueba no es valida

In [54]:
is_truth_a_leaf = verify(MT.get_root(), "truth", MT.get_proof_for("false"), test_hash)
print(is_truth_a_leaf)

False


Caso 3, valor (truth) no es una hoja.

In [55]:
is_not_a_leaf_a_leaf = verify(MT.get_root(), "not a leaf", MT.get_proof_for("truth"), test_hash)
print(is_not_a_leaf_a_leaf)

False
